In [1]:
from gpjax import RBF, Prior, Gaussian
import jax.numpy as jnp

In [2]:
x = jnp.linspace(-1., 1., 5).reshape(-1, 1)
y = jnp.sin(x)
p = Prior(RBF(), jitter=1e-6)
lik = Gaussian()
model = p*lik

In [3]:
model.neg_mll(x, y)

DeviceArray(6.28796596, dtype=float64)